In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import glob
import os
#pd.set_option('display.max_rows', 200)

In [2]:
data = pd.read_excel('..\\Datos Partidos\\Tigre_Situaciones.xlsx')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Session Start Date   28 non-null     datetime64[ns]
 1   Event                28 non-null     object        
 2   Session Name         28 non-null     object        
 3   Session Start        28 non-null     object        
 4   Session End          28 non-null     object        
 5   Tag Description      28 non-null     object        
 6   Tag Notes            0 non-null      float64       
 7   Tag Start            28 non-null     object        
 8   Tag End              28 non-null     object        
 9   Tag Duration (secs)  28 non-null     float64       
 10  Categoria            28 non-null     object        
 11  Zona2                28 non-null     float64       
 12  Zona                 28 non-null     float64       
 13  Accion               28 non-null     

In [3]:
data.columns = ['Fecha','etiquetado','name','Session Start','Session End','Evento','Tag Notes','Tag Start','Tag End','Tag Duration (secs)','Categoria','Zona inicial','Zona Final','Accion','Jugada']

In [4]:
data.sample(5)

,Fecha,etiquetado,name,Session Start,Session End,Evento,Tag Notes,Tag Start,Tag End,Tag Duration (secs),Categoria,Zona inicial,Zona Final,Accion,Jugada
6,2022-07-18,Tigre,Situaciones,10:15:27,12:27:53,En Contra,NaN,00:35:22,00:35:24,2000.0,ATAQUE COMBINADO,3.0,2.2,VOLVER,2.0
11,2022-07-18,Tigre,Situaciones,10:15:27,12:27:53,En Contra,NaN,00:45:43,00:45:45,2000.0,ATAQUE DIRECTO,2.2,2.1,REMATE ATAJADO,3.0
1,2022-07-18,Tigre,Situaciones,10:15:27,12:27:53,En Contra,NaN,00:28:20,00:28:22,2000.0,PELOTA PARADA,2.2,1.0,VOLVER,1.0
24,2022-07-18,Tigre,Situaciones,10:15:27,12:27:53,En Contra,NaN,01:31:14,01:31:16,2000.0,TRANSICION,2.2,2.2,REMATE ATAJADO,6.0
0,2022-07-18,Tigre,Situaciones,10:15:27,12:27:53,En Contra,NaN,00:28:13,00:28:15,2000.0,PELOTA PARADA,1.0,4.0,VOLVER,1.0


In [5]:
'''CORREGIR MAYUSCULAS Y COSAS DE EVENTOS'''

data['Evento'] = data['Evento'].str.lower().str.strip()
data['Categoria'] = data['Categoria'].str.lower().str.strip()
data['Accion'] = data['Accion'].str.lower().str.strip()

## Estadisticas 

In [6]:
data.groupby(['Evento','Categoria'])['Jugada'].nunique().to_frame('Cantidad')

Cantidad
Evento    Categoria                 
a favor   ataque combinado         1
          pelota parada            1
en contra ataque combinado         1
          ataque directo           2
          pelota parada            2
          transicion               1

### Armo las coordenadas

In [7]:
data['Zona inicial'] = data['Zona inicial'].replace('2A',2.1,regex=True)
data['Zona inicial'] = data['Zona inicial'].replace('2B',2.2,regex=True)
data['Zona inicial'] = data['Zona inicial'].replace('8A',8.1,regex=True)
data['Zona inicial'] = data['Zona inicial'].replace('8B',8.2,regex=True)
data['Zona Final'] = data['Zona Final'].replace('2A',2.1,regex=True)
data['Zona Final'] = data['Zona Final'].replace('2B',2.2,regex=True)
data['Zona Final'] = data['Zona Final'].replace('8A',8.1,regex=True)
data['Zona Final'] = data['Zona Final'].replace('8B',8.2,regex=True)

In [8]:
data['Zona inicial'].value_counts()

2.2    11
8.2     4
2.1     3
1.0     2
4.0     2
3.0     2
7.0     1
8.1     1
6.0     1
5.0     1
Name: Zona inicial, dtype: int64

In [9]:
data['Zona Final'].value_counts()

4.0    5
2.2    5
2.1    4
1.0    3
8.1    2
3.0    2
8.2    2
5.0    2
7.0    1
6.0    1
9.0    1
Name: Zona Final, dtype: int64

In [10]:
map_zona_coord_x = {  
1:45,
2.1:70,
2.2:30,
3:45,
4:136,
5:136,
6:136,
7:227,
8.1:255,
8.2:207,
9:227
}

In [11]:
map_zona_coord_y = {  
1:135,
2.1:81,
2.2:81,
3:27,
4:135,
5:81,
6:27,
7:135,
8.1:81,
8.2:81,
9:81
}

In [12]:
data['coord.final.num'] = np.where(data['Zona Final'] == 'volver',None, data['Zona Final']).astype('float')


In [13]:
data['Zona inicial x'] = data['Zona inicial'].map(map_zona_coord_x)
data['Zona Final x'] = data['coord.final.num'].map(map_zona_coord_x)
data['Zona inicial y'] = data['Zona inicial'].map(map_zona_coord_y)
data['Zona Final y'] = data['coord.final.num'].map(map_zona_coord_y)
data.iloc[:,-5:]

,coord.final.num,Zona inicial x,Zona Final x,Zona inicial y,Zona Final y
0,4.0,45,136,135,135
1,1.0,30,45,81,135
2,2.2,30,30,81,81
3,8.1,136,255,135,81
4,4.0,70,136,81,135
5,2.1,30,70,81,81
6,2.2,45,30,27,81
7,3.0,30,45,81,27
8,2.2,30,30,81,81
9,4.0,45,136,135,135


In [14]:
# pasar en ambos formatos. Crear las dos cols originales q sean strings

data['coord.inicial'] = round(data['Zona inicial x'],0).astype('str').str.split('.').str[0]+','+round(data['Zona inicial y'],0).astype('str').str.split('.').str[0]
data['coord.final'] = round(data['Zona Final x'],0).astype('str').str.split('.').str[0]+','+round(data['Zona Final y'],0).astype('str').str.split('.').str[0]


In [15]:
data['coord.inicial'] = np.where(data['coord.inicial']=='nan,nan', None, data['coord.inicial'])
data['coord.final'] = np.where(data['coord.final']=='nan,nan', None, data['coord.final'])

In [16]:
data[['coord.inicial','coord.final']]

,coord.inicial,coord.final
0,"45,135","136,135"
1,"30,81","45,135"
2,"30,81","30,81"
3,"136,135","255,81"
4,"70,81","136,135"
5,"30,81","70,81"
6,"45,27","30,81"
7,"30,81","45,27"
8,"30,81","30,81"
9,"45,135","136,135"


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Fecha                28 non-null     datetime64[ns]
 1   etiquetado           28 non-null     object        
 2   name                 28 non-null     object        
 3   Session Start        28 non-null     object        
 4   Session End          28 non-null     object        
 5   Evento               28 non-null     object        
 6   Tag Notes            0 non-null      float64       
 7   Tag Start            28 non-null     object        
 8   Tag End              28 non-null     object        
 9   Tag Duration (secs)  28 non-null     float64       
 10  Categoria            28 non-null     object        
 11  Zona inicial         28 non-null     float64       
 12  Zona Final           28 non-null     float64       
 13  Accion               28 non-null     

In [ ]:
data['Temporada'] = 'Temp 02-2022'
data['partido'] = 'Tigre'

In [ ]:
data.to_excel("../Datos generados/reserva02-22-fecha8_tigre_situaciones.xlsx",index=False)